In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import numpy as np
from sklearn.metrics import accuracy_score
from torch.utils.data import TensorDataset
import torchvision
from torchvision import models
from tqdm import tqdm
import pickle

/root/miniconda3/envs/CGAN/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/root/miniconda3/envs/CGAN/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [6]:
# get ID dataset
root = './../../data/mnist'  

trainset = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
])

testset = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
])

train_set = datasets.MNIST(root, train=True, download=True, transform=trainset)
test_set = datasets.MNIST(root, train=False, download=True, transform=testset)

batch_size = 3000
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False)

data_iter = iter(train_loader)
X_sub, _ = next(data_iter)
X_sub = X_sub.numpy()
print(X_sub)
print(len(X_sub))


[[[[-0.42421296 -0.42421296 -0.42421296 ... -0.42421296 -0.42421296
    -0.42421296]
   [-0.42421296 -0.42421296 -0.42421296 ... -0.42421296 -0.42421296
    -0.42421296]
   [-0.42421296 -0.42421296 -0.42421296 ... -0.42421296 -0.42421296
    -0.42421296]
   ...
   [-0.42421296 -0.42421296 -0.42421296 ... -0.42421296 -0.42421296
    -0.42421296]
   [-0.42421296 -0.42421296 -0.42421296 ... -0.42421296 -0.42421296
    -0.42421296]
   [-0.42421296 -0.42421296 -0.42421296 ... -0.42421296 -0.42421296
    -0.42421296]]]


 [[[-0.42421296 -0.42421296 -0.42421296 ... -0.42421296 -0.42421296
    -0.42421296]
   [-0.42421296 -0.42421296 -0.42421296 ... -0.42421296 -0.42421296
    -0.42421296]
   [-0.42421296 -0.42421296 -0.42421296 ... -0.42421296 -0.42421296
    -0.42421296]
   ...
   [-0.42421296 -0.42421296 -0.42421296 ... -0.42421296 -0.42421296
    -0.42421296]
   [-0.42421296 -0.42421296 -0.42421296 ... -0.42421296 -0.42421296
    -0.42421296]
   [-0.42421296 -0.42421296 -0.42421296 ... -0.

In [7]:
# get OOD KnockoffNet dataset 
root = './../../data/emnist' 
# 定义数据集的变换
trainset = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
])

testset = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
])

train_set = datasets.EMNIST(root, split='letters', train=True, download=True, transform=trainset)
test_set = datasets.EMNIST(root, split='letters', train=False, download=True, transform=testset)
batch_size = 3000
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False)
data_iter1 = iter(train_loader)
X_sub1, _ = next(data_iter1)
X_sub1 = X_sub1.numpy()
print(X_sub1)
print(len(X_sub1))

[[[[-0.42421296 -0.42421296 -0.42421296 ... -0.42421296 -0.42421296
    -0.42421296]
   [-0.42421296 -0.42421296 -0.42421296 ... -0.42421296 -0.42421296
    -0.42421296]
   [-0.42421296 -0.42421296 -0.42421296 ... -0.42421296 -0.42421296
    -0.42421296]
   ...
   [-0.42421296 -0.42421296 -0.42421296 ... -0.42421296 -0.42421296
    -0.42421296]
   [-0.42421296 -0.42421296 -0.42421296 ... -0.42421296 -0.42421296
    -0.42421296]
   [-0.42421296 -0.42421296 -0.42421296 ... -0.42421296 -0.42421296
    -0.42421296]]]


 [[[-0.42421296 -0.42421296 -0.42421296 ... -0.42421296 -0.42421296
    -0.42421296]
   [-0.42421296 -0.42421296 -0.42421296 ... -0.42421296 -0.42421296
    -0.42421296]
   [-0.42421296 -0.42421296 -0.42421296 ... -0.42421296 -0.42421296
    -0.42421296]
   ...
   [-0.42421296 -0.42421296 -0.42421296 ... -0.42421296 -0.42421296
    -0.42421296]
   [-0.42421296 -0.42421296 -0.42421296 ... -0.42421296 -0.42421296
    -0.42421296]
   [-0.42421296 -0.42421296 -0.42421296 ... -0.

In [20]:

concat_data = np.concatenate((X_sub, X_sub1), axis=0)
#print(concat_data)
print(len(concat_data))


6000


In [21]:
import numpy as np

# get OOD JBDA dataset 
loaded_data = np.load('../../mnist/X_sub1500.npy')
print("原始数据长度:", len(loaded_data))
print("原始数据形状:", loaded_data.shape)

# 获取后1500个数据
last_1500_data = loaded_data[-1500:]
print("后1500个数据的长度:", len(last_1500_data))
print("后1500个数据的形状:", last_1500_data.shape)


原始数据长度: 3000
原始数据形状: (3000, 1, 28, 28)
后1500个数据的长度: 1500
后1500个数据的形状: (1500, 1, 28, 28)


In [22]:
# 连接数组
concat_data = np.concatenate((concat_data, last_1500_data), axis=0)
#print(concat_data)
print(len(concat_data))


7500


In [23]:
# 创建全零标签数组
labels_sub = np.zeros(concat_data.shape[0])
# 将concat_data分成三个部分，并为每个部分赋予相应的标签
labels_sub[:3000] = 0  # 前1500个元素的标签为0
labels_sub[3000:6000] = 1  # 接下来的1500个元素的标签为1
labels_sub[6000:] = 2  # 最后1500个元素的标签为2
#labels_sub[4500:] = 3

# 打印标签数组的长度和0形状
print(len(labels_sub))
print(labels_sub.shape)

7500
(7500,)


In [24]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
# 转换为PyTorch的Tensor对象
X_tensor = torch.from_numpy(concat_data).float()
y_tensor = torch.from_numpy(labels_sub).long()


# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)
print(X_train, X_test, y_train, y_test)
print("训练集长度:", len(X_train))
print("测试集长度:", len(X_test))
print("训练标签长度:", len(y_train))
print("测试标签长度:", len(y_test))

print(X_train.shape)
print(y_train.shape)

tensor([[[[-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          ...,
          [-0.4242, -0.4242, -0.4115,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242]]],


        [[[-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          ...,
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242]]],


        [[[-0.2242, -0.2242, -0.2242,  ..., -0.6242, -0.6242, -0.4242],
          [-0.2242, -0.224

In [25]:
class MultiClassifier(nn.Module):
    def __init__(self):
        super(MultiClassifier,self).__init__()
        self.conv1 = nn.Conv2d(1,32,3,padding=1)
        self.conv2 = nn.Conv2d(32,64,3,padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu1 = nn.ReLU()

        self.conv3 = nn.Conv2d(64,128,3,padding=1)
        self.conv4 = nn.Conv2d(128, 128, 3,padding=1)
        self.pool2 = nn.MaxPool2d(2, 2, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.relu2 = nn.ReLU()

        self.conv5 = nn.Conv2d(128,128, 3,padding=1)
        self.conv6 = nn.Conv2d(128, 128, 3,padding=1)
        self.conv7 = nn.Conv2d(128, 128, 1,padding=1)
        self.pool3 = nn.MaxPool2d(2, 2, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.relu3 = nn.ReLU()

        self.conv8 = nn.Conv2d(128, 256, 3,padding=1)
        self.conv9 = nn.Conv2d(256, 256, 3, padding=1)
        self.conv10 = nn.Conv2d(256, 256, 1, padding=1)
        self.pool4 = nn.MaxPool2d(2, 2, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.relu4 = nn.ReLU()

        self.conv11 = nn.Conv2d(256, 512, 3, padding=1)
        self.conv12 = nn.Conv2d(512, 512, 3, padding=1)
        self.conv13 = nn.Conv2d(512, 512, 1, padding=1)
        self.pool5 = nn.MaxPool2d(2, 2, padding=1)
        self.bn5 = nn.BatchNorm2d(512)
        self.relu5 = nn.ReLU()

        self.fc14 = nn.Linear(512*4*4,1024)
        self.drop1 = nn.Dropout2d()
        self.fc15 = nn.Linear(1024,1024)
        self.drop2 = nn.Dropout2d()
        self.fc16 = nn.Linear(1024,3)


    def forward(self,x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.pool1(x)
        x = self.bn1(x)
        x = self.relu1(x)


        x = self.conv3(x)
        x = self.conv4(x)
        x = self.pool2(x)
        x = self.bn2(x)
        x = self.relu2(x)

        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = self.pool3(x)
        x = self.bn3(x)
        x = self.relu3(x)

        x = self.conv8(x)
        x = self.conv9(x)
        x = self.conv10(x)
        x = self.pool4(x)
        x = self.bn4(x)
        x = self.relu4(x)

        x = self.conv11(x)
        x = self.conv12(x)
        x = self.conv13(x)
        x = self.pool5(x)
        x = self.bn5(x)
        x = self.relu5(x)
        # print(" x shape ",x.size())
        x = x.view(-1,512*4*4)
        x = F.relu(self.fc14(x))
        x = self.drop1(x)
        x = F.relu(self.fc15(x))
        x = self.drop2(x)
        x = self.fc16(x)

        return x

In [26]:
# 定义逻辑回归模型
num_classes = 3
#model = MultiClassifier()
model = MultiClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
# 将数据和标签送入设备（如GPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_train = X_train.to(device)
y_train = y_train.to(device)
model = model.to(device)
# 创建训练数据加载器
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# 创建测试数据加载器
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# 假设您的模型已经定义为 model
# 假设您的数据加载器已经定义为 train_loader
# 假设您的设备已经定义为 device

num_epochs = 55
model.train()

# 使用 Adam 优化器和 L2 正则化
optimizer = optim.SGD(model.parameters(), lr=0.001, weight_decay=1e-5)

# 交叉熵损失函数
criterion = nn.CrossEntropyLoss()

# 学习率调度器
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        # 前向传播
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:    # 每100个批次打印一次
            print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 100:.4f}")
            running_loss = 0.0

    # 更新学习率
    scheduler.step()

# 保存训练好的模型参数

torch.save(model.state_dict(), './mnist/attackmnist3.pth')

print('训练完成')


In [27]:
# 在测试集上评估模型性能
model.load_state_dict(torch.load('./mnist/attackmnist3.pth'))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()
test_loss = 0.0
correct = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)    
        outputs = model(inputs)
#       print(outputs)
        _, predicted = torch.max(outputs, 1)
#        print(predicted)
#        print(labels)
        if predicted == labels:
            correct += 1
    test_accuracy = correct / len(test_dataset)
print(f'Test Accuracy: {test_accuracy:.4f}')


Test Accuracy: 0.9953


In [28]:
# 初始化计数器
TP = 0
FP = 0
TN = 0
FN = 0
model.load_state_dict(torch.load('./mnist/attackmnist3.pth'))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()
# 在测试集上评估模型性能
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)    
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)

        # 更新计数器
        for label, prediction in zip(labels, predicted):
            if label == 0 and prediction == 0:
                TP += 1
            elif label != 0 and prediction == 0:
                FP += 1
            elif label == 0 and prediction != 0:
                FN += 1
            elif label != 0 and prediction != 0:
                TN += 1

# 计算FPR和FNR
FPR = FP / (FP + TN) if (FP + TN) > 0 else 0
FNR = FN / (FN + TP) if (FN + TP) > 0 else 0

print(f"FPR: {FPR:.4f}")
print(f"FNR: {FNR:.4f}")


FPR: 0.0069
FNR: 0.0016


In [30]:
TP = 0  # 真阳性
FN = 0  # 假阴性
model.load_state_dict(torch.load('./mnist/attackmnist3.pth'))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)

        for label, prediction in zip(labels, predicted):
            if label == 2 and prediction == 2:
                TP += 1  # 真阳性：正确预测为类别1
            elif label == 2 and prediction != 2:
                FN += 1  # 假阴性：错误预测为非类别1

# 计算 FNR
FNR = FN / (FN + TP) if (FN + TP) > 0 else 0
print(f"FNR for class 2: {FNR:.4f}")


FNR for class 2: 0.0000


In [31]:
root = './../../data/mnist'  # 数据集存储的根目录

# 定义数据集的变换
trainset = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
])

testset = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
])

# 加载数据集
train_set = datasets.MNIST(root, train=True, download=True, transform=trainset)
test_set = datasets.MNIST(root, train=False, download=True, transform=testset)

# 定义 DataLoader
batch_size = 1
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False)

IN_count = 0
JBDA_count = 0
Knockoff_count = 0
input_count = 0

# 修改模型的全连接层
num_classes = 3  # 新的类别数量
model = MultiClassifier()
model.load_state_dict(torch.load('./mnist/attackmnist3.pth'))
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
model = model.to(device)
# 在测试集上评估模型性能
model.eval()
with torch.no_grad():
    for inputs, _ in tqdm(test_loader, desc="Validation Progress"):
        input_count += 1
        inputs = inputs.to(device)
        outputs = model(inputs)
#        print(outputs)
        _, predicted = torch.max(outputs, 1)
        if predicted == 0 :
            IN_count += 1
        elif predicted == 1 :
            Knockoff_count += 1
        elif predicted == 2:
            JBDA_count += 1
    print(f"input: {input_count:}")
    print(f"IN: {IN_count:}")
    print(f"Knockoff Data: {Knockoff_count:}")
    print(f"JBDA Data: {JBDA_count:}")

Validation Progress: 100%|████████████████████████████████████| 10000/10000 [00:17<00:00, 577.75it/s]

input: 10000
IN: 9978
Knockoff Data: 22
JBDA Data: 0


In [32]:
root = './../../data/emnist'  # 数据集存储的根目录
# 定义数据集的变换
trainset = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
])

testset = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
])

train_set = datasets.EMNIST(root, split='letters', train=True, download=True, transform=trainset)
test_set = datasets.EMNIST(root, split='letters', train=False, download=True, transform=testset)
batch_size = 3000
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False)
# 定义 DataLoader
batch_size = 1
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False)

IN_count = 0
JBDA_count = 0
Knockoff_count = 0
input_count = 0

# 修改模型的全连接层
num_classes = 3  # 新的类别数量
model = MultiClassifier()
model.load_state_dict(torch.load('./mnist/attackmnist3.pth'))
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
model = model.to(device)
# 在测试集上评估模型性能
model.eval()
with torch.no_grad():
    for inputs, _ in tqdm(test_loader, desc="Validation Progress"):
        input_count += 1
        inputs = inputs.to(device)
        outputs = model(inputs)
#        print(outputs)
        _, predicted = torch.max(outputs, 1)
        if predicted == 0 :
            IN_count += 1
        elif predicted == 1 :
            Knockoff_count += 1
        elif predicted == 2:
            JBDA_count += 1
    print(f"input: {input_count:}")
    print(f"IN: {IN_count:}")
    print(f"Knockoff Data: {Knockoff_count:}")
    print(f"JBDA Data: {JBDA_count:}")

Validation Progress: 100%|████████████████████████████████████| 20800/20800 [00:36<00:00, 571.13it/s]

input: 20800
IN: 125
Knockoff Data: 20673
JBDA Data: 2
